- Julia version: 1.10
- Author: Marcin Latawiec (311031)
- Date: 2024-06-22

In [2]:
include("../src/Graph.jl")
include("../src/DataModule.jl")
include("../src/UtilsModule.jl")
include("../src/AccuracyModule.jl")
include("../src/GradientOptimizersModule.jl")

Main.GradientOptimizersModule

In [3]:
using .DataModule, .UtilsModule, .AccuracyModule
using Random, Plots

In [4]:
function load_data(batch_size)
    println("Loading train data...")
    train_x, train_y = DataModule.preprocess(:train; one_hot = true)

    train_x_batched = DataModule.batch(train_x, batch_size)
    train_y_batched = DataModule.batch(train_y, batch_size)

    println("Loading test data...")
    test_x, test_y = DataModule.preprocess(:test; one_hot = true)
    return train_x, train_y, train_x_batched, train_y_batched, test_x, test_y
end

load_data (generic function with 1 method)

In [ ]:
function update_weights!(graph::Vector, optimizer::GradientOptimizersModule.GradientOptimizer)
    for node in graph
        if isa(node, Variable)
            if node.gradient != nothing
                node.output .-= optimizer(node.gradient)
                node.gradient .= 0
            elseif node.gradient == nothing
                node.output = nothing
            end
        end
    end
end

Parametryzowana architektura sieci

In [7]:
function main(batch_size, epochs, optimizer)
    train_x, train_y, train_x_batched, train_y_batched, test_x, test_y = load_data(batch_size)

    x = Variable([0.], name="x")

    wd = Variable(UtilsModule.glorot_uniform(10, 64))
    bd = Variable(UtilsModule.glorot_uniform(10, ))
    fd = Constant(UtilsModule.identity)
    dfd = Constant(UtilsModule.identity_deriv)

    wr = Variable(UtilsModule.glorot_uniform(64, 196))
    br = Variable(UtilsModule.glorot_uniform(64, ))
    hwr = Variable(UtilsModule.glorot_uniform(64, 64))
    states = Variable(nothing, name = "states")
    fr = Constant(tanh)
    dfr = Constant(UtilsModule.tanh_deriv)

    optimizer = GradientOptimizersModule.Descent(15e-3)

    r = rnn_layer(x, wr, br, hwr, states, fr, dfr)
    d = dense_layer(r, wd, bd, fd, dfd)
    graph = topological_sort(d)


    batch_loss = Float64[]
    batch_acc = Float64[]
    println("Training")
    for epoch in 1:epochs
        batches = randperm(size(train_x_batched, 1))
        @time for batch in batches
            states.output = nothing
            x.output = train_x_batched[batch][  1:196,:]
            forward!(graph)

            x.output = train_x_batched[batch][197:392,:]
            forward!(graph)

            x.output = train_x_batched[batch][393:588,:]
            forward!(graph)

            x.output = train_x_batched[batch][589:end,:]
            result = forward!(graph)

            loss = AccuracyModule.loss(result, train_y_batched[batch])
            acc = AccuracyModule.accuracy(result, train_y_batched[batch])
            push!(batch_loss, loss)
            push!(batch_acc, acc)
            
            gradient = AccuracyModule.gradient(result, train_y_batched[batch]) ./ batch_size
            backward!(graph, seed=gradient)
            update_weights!(graph, optimizer)
        end
        states.output = nothing
        test_graph = topological_sort(d)

        x.output = test_x[  1:196,:]
        forward!(test_graph)

        x.output = test_x[197:392,:]
        forward!(test_graph)

        x.output = test_x[393:588,:]
        forward!(test_graph)

        x.output = test_x[589:end,:]
        result = forward!(test_graph)

        loss = AccuracyModule.loss(result, test_y)
        acc = AccuracyModule.accuracy(result, test_y)

        @show epoch loss acc
    end
    return batch_loss, batch_acc
end

main (generic function with 1 method)

In [8]:
batch_loss, batch_acc = main(100, 5, GradientOptimizersModule.Descent(15e-3))

Loading train data...
Loading test data...
Training


LoadError: MethodError: no method matching Vector{Float64}(::Matrix{Float32})

[0mClosest candidates are:
[0m  Array{T, N}(::AbstractArray{S, N}) where {T, N, S}
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:673[24m[39m
[0m  Array{T, N}([91m::Missing[39m, Any...) where {T, N}
[0m[90m   @[39m [90mBase[39m [90m[4mbaseext.jl:43[24m[39m
[0m  Vector{T}([91m::UndefInitializer[39m, [91m::Tuple{Int64}[39m) where T
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:486[24m[39m
[0m  ...
